In [22]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_parquet('_df_for_modeling.parquet')

In [33]:
df['host_is_superhost']

0       f
1       f
2       t
3       f
4       f
       ..
6858    t
6859    f
6860    t
6861    f
6862    f
Name: host_is_superhost, Length: 6863, dtype: object

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6863 entries, 0 to 6862
Data columns (total 73 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6863 non-null   int64  
 1   listing_url                                   6863 non-null   object 
 2   scrape_id                                     6863 non-null   int64  
 3   last_scraped                                  6863 non-null   object 
 4   name                                          6863 non-null   object 
 5   description                                   6850 non-null   object 
 6   neighborhood_overview                         4913 non-null   object 
 7   picture_url                                   6863 non-null   object 
 8   host_id                                       6863 non-null   int64  
 9   host_url                                      6863 non-null   o

In [28]:
len(df.columns)

73

In [36]:
def data_cleanup_parquet(df):

    df.dropna(how='all', axis=1, inplace=True) #drop empty cols

    number_looking_strings = ['id', 'scrape_id', 'host_id']
    for col in number_looking_strings:
        df[col] = df[col].astype("string")

    df['host_response_rate'] = df['host_response_rate'].str.replace("%","").astype(float)/100 #previously a percentage as a string
    df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace("%","").astype(float)/100 #previously a percentage as a string
    df['price'] = df['price'].str.replace(r'[$,]',"").astype(float) #previosuly represented as a string

    date_cols = ['last_scraped', 'host_since', 'calendar_last_scraped', 'first_review', 'last_review']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], format=r'%Y-%m-%d')

    for col in df.columns:
        if df[col][0] == "f" or df[col][0] == "t": #if first record is "t" or "f" we're going to convert to bool - still should spot check after just incase first record of bool is null
            df[col] = df[col].str.replace("f", "0").astype('string')
            df[col] = df[col].str.replace("t", "1").astype('string')
            print(df[col])
            try: ## int can't hold nulls so if the conversion fails due to a value error, convert to float instead
                df[col] = df[col].astype(int)
            except ValueError:
                df[col] = df[col].astype(float)


    remaining_string_cols = df.select_dtypes(include=["object"]).columns #all remaining object columns are strings so I'm explicitly setting them to be
    for col in remaining_string_cols:
        df[col] = df[col].astype('string')

    list_string_cols = ['host_verifications', 'amenities'] #these 2 columns are string lists so I'm converting them back to object just so I remember to handle them differently
    for col in list_string_cols:
        df[col] = df[col].astype('object')

    return df

df_clean = data_cleanup_parquet(df)

AttributeError: Can only use .str accessor with string values!